In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive")
os.mkdir("NLP-AIBOT")


FileExistsError: ignored

In [ ]:
import os
os.chdir("/content/drive/My Drive/NLP-AIBOT")

In [ ]:
df_weather = pd.read_csv('./Dataset - Weather.csv', encoding = 'utf-8')
df_religous = pd.read_csv('Dataset - Religious_time.csv', encoding = 'utf-8')
df_time = pd.read_csv('./Dataset - Time.csv', encoding = 'utf-8')
df_cal = pd.read_csv('./Dataset - Calendar.csv', encoding = 'utf-8')
df_greeting = pd.read_csv('./Dataset - Greeting.csv', encoding = 'utf-8')


In [ ]:
df_greeting.dropna(subset = ['Sentences'], inplace=True)

In [ ]:
df_greeting.columns

Index(['Sentences', 'Answer', 'response type', 'intent'], dtype='object')

In [ ]:
df_weather = df_weather.drop([0])
df_time = df_time.drop([0])
df_religous = df_religous.drop([0])
df_cal = df_cal.drop([0])
df_greeting = df_greeting.drop([9])

In [ ]:
df_weather.columns

Index(['Unnamed: 0', 'command', 'command details', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'api details', 'Unnamed: 12', 'Unnamed: 13',
       'response details', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'response'],
      dtype='object')

In [ ]:
command_weather = []
command_time = []
command_cal = []
command_religous = []
command_greeting = []

for command, intent in zip(df_weather['command'], df_weather['command details']):
    temp = []
    temp.append(command)
    temp.append(0)
    command_weather.append(temp)

for command, intent in zip(df_time['command'], df_time['command details']):
    temp = []
    temp.append(command)
    temp.append(1)
    command_time.append(temp)

for command, intent in zip(df_religous['command'], df_religous['command details']):
    temp = []
    temp.append(command)
    temp.append(2)
    command_religous.append(temp)

for command, intent in zip(df_cal['command'], df_cal['command details']):
    temp = []
    temp.append(command)
    temp.append(3)
    command_cal.append(temp)

for command in df_greeting['Sentences']:
    temp = []
    temp.append(command)
    temp.append(4)
    command_greeting.append(temp)
    

In [ ]:
df_weather['command'][10]

'اندازه دمای هوای دو روز بعد تهران به چه میزان است'

In [ ]:
command_weather_train = []
command_time_train = []
command_cal_train = []
command_religous_train = []
command_greeting_train = []

command_weather_test = []
command_time_test = []
command_cal_test = []
command_religous_test = []
command_greeting_test = []

for index in range(len(df_weather['command'])):
    t1 =int(len(df_weather['command']) * 0.9)
    if index <= t1:
      command_weather_train.append([df_weather['command'][index], 0])
    else:
      command_weather_test.append([df_weather['command'][index], 0])

for index in range(len(df_weather['command'])):
    t1 =int(len(df_weather['command']) * 0.9)
    if index <= t1:
      command_time_train.append([df_weather['command'][index], 1])
    else:
      command_time_test.append([df_weather['command'][index], 1])

for command, intent in zip(df_religous['command'], df_religous['command details']):
    temp = []
    temp.append(command)
    temp.append(2)
    command_religous.append(temp)

for command, intent in zip(df_cal['command'], df_cal['command details']):
    temp = []
    temp.append(command)
    temp.append(3)
    command_cal.append(temp)

for command in df_greeting['Sentences']:
    temp = []
    temp.append(command.strip())
    temp.append(4)
    command_greeting.append(temp)
    

In [ ]:
len(command_weather), len(command_cal), len(command_religous), len(command_time), len(command_greeting)

(133, 40, 30, 26, 33)

In [ ]:
dataset = [*command_weather, *command_religous, *command_time, *command_cal, *command_greeting]

In [ ]:
len(dataset)

262

In [ ]:
import pickle
with open('dataset.pkl', 'wb') as f:
  pickle.dump(dataset, f)

In [ ]:
favorite_color = pickle.load( open( "dataset.pkl", "rb" ) )

In [ ]:
favorite_color[200:]

[['روز جمهوری اسلامی در سال 1400', 3],
 ['کدام روز از سال ۱۴۰۰ روز قلم است', 3],
 ['سالگرد اعلام پذیرش قطعنامه ۵۹۸ شورای امنیت از سوی ایران در کدام روز ۱۴۰۰ اتفاق افتاده است',
  3],
 ['تاریخ روز صنعت و معدن امسال چه روزی است', 3],
 ['در این سال روز عرفه چه روزی بود', 3],
 ['تاریخ برگزاری اولین کنکور در ایران', 3],
 ['روز خبرنگار کدام روز از امسال است', 3],
 ['عید سعید غدیر خم کدام روز امسال است', 3],
 ['تاریخ ولادت امام موسی کاظم علیه السلام در سال ۹۹', 3],
 ['تاریخ روز جهانی چپ دست ها در سال 99', 3],
 ['روز بزرگداشت ابوعلی سینا و روز پزشک در کدام روز سال ۹۹ است', 3],
 ['روز سینما در سال ۱۳۹۹', 3],
 ['مناسبت روز ۱۸ اسفند امسال چیست', 3],
 ['مناسبت ۱۸ اسفند سال بعد چیست', 3],
 ['چه مناسبت هایی در هفته پیش رو وجود دارد', 3],
 ['چه مناسبت هایی در تاریخ 14 تا 19 دی وجود دارد ', 3],
 ['22 بهمن سال آینده به میلادی؟', 3],
 ['17 شهریور به میلادی؟', 3],
 ['تاریخ میلادی امروز', 3],
 ['11 محرم سال بعد به میلادی؟', 3],
 ['12 فوریه 1996 به شمسی؟ ', 3],
 ['تاریخ شمسی امروز', 3],
 ['21 رمضان 1449 به 

In [ ]:
sentences = []
intents = []
for text, i in dataset:
    sentences.append(text)
    intents.append(i)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)


In [ ]:
total_words = len(tokenizer.word_index) + 1
total_words

342

In [ ]:
input_sequences = tokenizer.texts_to_sequences(sentences)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post'))
input_sequences[0]

array([ 52, 112,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
label = ku.to_categorical(intents, num_classes=5)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length= max_sequence_len - 1))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(5, activation='softmax'))


In [ ]:
# Pick an optimizer
adam = Adam(learning_rate = 0.01)
model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics = ["accuracy"])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 19, 100)           34200     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 119,325
Trainable params: 119,325
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
 history = model.fit(input_sequences, label, epochs=100, verbose=1)

Epoch 1/100
9/9 [==============================] - 3s 21ms/step - loss: 1.1048 - accuracy: 0.5191
Epoch 2/100
9/9 [==============================] - 0s 23ms/step - loss: 0.5373 - accuracy: 0.7824
Epoch 3/100
9/9 [==============================] - 0s 21ms/step - loss: 0.1796 - accuracy: 0.9809
Epoch 4/100
9/9 [==============================] - 0s 21ms/step - loss: 0.1698 - accuracy: 0.9618
Epoch 5/100
9/9 [==============================] - 0s 25ms/step - loss: 0.0284 - accuracy: 0.9962
Epoch 6/100
9/9 [==============================] - 0s 23ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 7/100
9/9 [==============================] - 0s 22ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 8/100
9/9 [==============================] - 0s 21ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 9/100
9/9 [==============================] - 0s 24ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 10/100
9/9 [==============================] - 0s 23ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 11/100
9/9 [=

In [ ]:
test_text = "اذان صبح تهران ساعت چند است"
token_list = tokenizer.texts_to_sequences([test_text])
token_list = pad_sequences(token_list, maxlen=max_sequence_len, padding='post')
token_list


array([[14, 28,  4, 17,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [ ]:
model.save('nlp_model.h5')

In [ ]:
model2 = Sequential()
model2.add(Embedding(301, 100, input_length= max_sequence_len - 1))
model2.add(Bidirectional(LSTM(64)))
model2.add(Dense(4, activation='softmax'))

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.load_model('nlp_model.h5')

In [ ]:
predicted = model.predict_classes(token_list, verbose= 0)
predicted[0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


2

In [ ]:
dict_thereshold = []

for sentences, intent in dataset:

  token_list = tokenizer.texts_to_sequences([sentences])
  token_list = pad_sequences(token_list, maxlen=max_sequence_len, padding='post')
  predicted = model.predict(token_list)
  t1 = np.argmax(predicted)
  t2 = predicted[0][t1]
  dict_thereshold.append([sentences, t1, t2])


In [ ]:
dict_thereshold[0]

[0, 0.9999956]

In [ ]:
for i in dict_thereshold:
  print(i)

['هوا چطور است', 0, 0.9998723]
['امروز هوا چطور است', 0, 0.9999763]
['دمای هوای امروز چقدر است', 0, 0.9999945]
['دمای هوای مشهد فردا چقدر است', 0, 0.99999726]
['دمای هوای چهار روز بعد آبادان چقدر است', 0, 0.9999975]
['دمای هوای دو روز بعد گرگان چند درجه سانتیگراد است', 0, 0.99999774]
['درجه گرمای هوای دو روز بعد پاریس', 0, 0.9999957]
['دمای هوای امروز مسکو', 0, 0.9999181]
['اندازه ی گرمی یا سردی دمای امروز توکیو', 0, 0.9999962]
['اندازه دمای هوای دو روز بعد تهران به چه میزان است', 0, 0.99999726]
['میانگین دمای هوای تهران امروز چقدر است', 0, 0.99999654]
['میانگین دمای هوای مشهد فردا چقدر است', 0, 0.9999974]
['میانگین اندازه گرمی یا سردی هوای دو روز بعد کاشان چند درجه است', 0, 0.9999976]
['دمای هوای چهار روز بعد پایتخت انگلستان چقدر است', 0, 0.99999774]
['دمای هوای دو روز بعد پایتخت آذربایجان چند درجه سانتیگراد است', 0, 0.99999785]
['دمای هوای فردای شیراز ساعت 14:25 چقدر است', 0, 0.9999974]
['دمای هوای پس فردای قزوین ساعت ۱۶:۳۲ چقدر است', 0, 0.9999975]
['دمای هوای دو روز بعد زنجان ساعت ۱

salam
